# Open Modelica - Transformation Matrix Example

Demonstrates the use of Modelica.MultiBody.Frames.axesRotations function.  Basic documentation on this function is available here:
http://doc.modelica.org/om/Modelica.Mechanics.MultiBody.Frames.TransformationMatrices.html

You can edit the MODELICAPATH and/or the OPENMODELICALIBRARY environment variables in Windows to include the path to where all of your OpenModelica models are stored.  When you open OMedit, this will pre-load all model files in your directory.

I am using the Jupyter Notebook with an OpenModelica kernel (essentially the OMshell in Notebook form).  The following commands are similar to what you would enter in the OMshell window.  Here I don't specify the path to these libraries because I've added the directory to my MODELICPATH variable (I added it to my user account environment variable and to the system environment variable just to be sure).  I also used the forward slash `/` and not the usual Windows back slash `\`.

So first I load my satellite models from P1 and then the Modelica library.

In [33]:
loadModel(SatModel)

True

In [34]:
loadModel(Modelica)

True

Here I have changed the nature of the `sat_ECI` function a bit from what is requested in P2.  In order to use the `axesRotations` and later the `resolve2` Modelica functions, I have to create 1-d arrays in OpenModelica (note NOT the `Vector` record).  This is because these functions take generic arrays and not specific `Vector` variables.

In [72]:
within SatModel;
function sat_ECI
    import Modelica.Mechanics.MultiBody.Frames.
    TransformationMatrices.axesRotations;
    
    input Real ang[3] "-argper, -inc, -raan (rad)";
    output Real TM[3, 3];
protected  
    Integer seq[3] = {3,1,3} "Angle sequence from pf to ECI";
    
algorithm
       TM := axesRotations(sequence=seq, angles=ang);
    
end sat_ECI;

('sat_ECI.SatModel',)

Note a few other things:
  1. The transformation matrix from Peri-focal to ECI requires backwards rotations always about the {3, 1, 3} axes.  Over the course of a tracking period (of order 30 minutes), this transformation will not change appreciably, so its the same one for the full trajectory - this is NOT true of the `sat_ECF` transformation matrix which will change slightly with time.
  2.  I am using the `within` keyword - can be used to add models to a virtual library.  Allows you to have separate files for different models but integrated in a single library.  There is some set up involved with this - see http://book.xogeny.com/components/packages/package_def/ for details.
  3.  In this case the `sat_ECI` function outputs the transformation matrix from perifocal to ECI coordinates.  I will then use it to calculate the ECI coordinates in my calling model.

Now I need to instantiate these models in a simulation (i.e. I need a calling model).  In this case I bring in the data for the PRN13 spacecraft I have been using in previous examples.

I instantiate the `SatModel.Satellite` model which calculates the perifocal coordinate of the spacecraft.  I chose a value of `tstart = 86400` (1 day) to check the accuracy of the propagation.  I then calculate the ECI coordinates using the transformation matrix from the `sat_ECI` function.

In [58]:
model GPS_prn13
  constant Real d2r = Modelica.Constants.D2R;
  parameter Real rotang[3] = 
                           {-93.2441*d2r, -55.5066*d2r, -217.3716*d2r};
  import 
   Modelica.Mechanics.MultiBody.Frames.TransformationMatrices.resolve2;
  SatModel.Satellite prn13(M0=267.123, N0=2.00565, ecc=0.0034407,
                     Ndot2=0.00000068, Nddot6=0., tstart=86400.);
  Real TM[3,3] = SatModel.sat_ECI(ang=rotang);
  Real p_sat_ECI[3];
  Real v_sat_ECI[3];
  
equation
    p_sat_ECI = resolve2(TM, prn13.p_sat_pf);
    v_sat_ECI = resolve2(TM, prn13.v_sat_pf);
  
end GPS_prn13;

('GPS_prn13',)

Now to simulate and plot. I am assuming simulation time `0` is the start of the tracking period.  I plot the Perifocal and ECI coordinates vs time...

In [59]:
simulate(GPS_prn13,stopTime=1800., numberOfInterval=30.)

{'messages': '', 'timeBackend': 0.0994154309030959, 'timeCompile': 11.54396386716399, 'timeTemplates': 0.0884811585118179, 'resultFile': 'c:/Notebooks/ENG4350/GPS_prn13_res.mat', 'timeFrontend': 0.8858825122300034, 'timeSimCode': 0.01708233349215002, 'timeSimulation': 0.356825571513102, 'simulationOptions': "startTime = 0.0, stopTime = 1800.0, numberOfIntervals = 500, tolerance = 1e-006, method = 'dassl', fileNamePrefix = 'GPS_prn13', options = '', outputFormat = 'mat', variableFilter = '.*', cflags = '', simflags = ''", 'timeTotal': 12.99226706151101}

In [60]:
plot({prn13.p_sat_pf[1], prn13.p_sat_pf[2]})

Parametric plots are in an external (OMPlot) window...

In [67]:
plotParametric(prn13.p_sat_pf[1], prn13.p_sat_pf[2])

True

In [68]:
plot({prn13.v_sat_pf[1], prn13.v_sat_pf[2]})

Here is the DCM between Peri-focal and ECI coordinates calculated by OpenModelica
Compare with Data component generated in STK

In [69]:
plot({TM[1,1], TM[2,1], TM[3,1], TM[1,2], TM[2,2], TM[3,2],TM[1,3],TM[2,3],TM[3,3]})

Finally here are the satellite position and velocity vectors in ECI coordinates...

In [73]:
plot({p_sat_ECI[1], p_sat_ECI[2], p_sat_ECI[3]})

In [74]:
plot({v_sat_ECI[1], v_sat_ECI[2], v_sat_ECI[3]})